### Contents

* Guidelines for Prompting
  - Write clear and specific instructions 
  - Give the model time to "think"
  
* Iterative prompt development
* Capabilities 
  - Summarizing
  - **Infering**
  - Transforming
  - Expanding
* Building a Chatbot

# Inferring

이 수업에서는 제품 리뷰 및 뉴스 기사에서 감정 및 주제를 추론합니다.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 이것는 모델 출력의 무작위성 정도를 말합니다.[0~1]
    )
    return response.choices[0].message.content

## Product review text

In [3]:
lamp_review = """
내 침실을 위한 멋진 램프가 필요했고, 이 램프는 추가 저장 공간이 있고\
가격대가 너무 높지 않았습니다. 빨리 받았습니다. 운송 중에 우리 램프의 줄이\
끊어졌고 회사는 기꺼이 새 램프를 보냈습니다. 그것도 몇일 만에 왔습니다.\
조립하기 쉬웠습니다. 부품이 없어서 지원팀에 연락했더니 아주 빨리\
잃어버린 부품을 받았습니다! 루미나는 고객과 제품을 생각하는\
좋은 회사인 것 같아요!!
"""

## Sentiment (positive/negative)

In [4]:
prompt = f"""
세 개의 백틱으로 구분된 다음 제품 리뷰의 감정은 무엇입니까?

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

긍정적인 감정을 표현하고 있습니다.


In [5]:
prompt = f"""
세 개의 백틱으로 구분된 다음 제품 리뷰의 감정은 무엇입니까?

"긍정적" 또는 "부정적" 중 한 단어로 대답하십시오.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

긍정적


## Identify types of emotions

In [6]:
prompt = f"""
다음 리뷰 작성자가 표현하고 있는 감정의 목록을 확인합니다.\
목록에 5개 이하의 항목을 포함하십시오. \
쉼표로 구분된 단어 목록으로 답변 형식을 지정합니다.

리뷰 텍스트:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

만족, 빠른, 친절, 편리, 고객서비스


## Identify anger

In [7]:
prompt = f"""
다음 리뷰의 작성자가 분노를 표현하고 있습니까?\
리뷰는 세 개의 백틱으로 구분됩니다.\
예 또는 아니오로 대답하십시오.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

아니오.


## Extract product and company name from customer reviews

In [8]:
prompt = f"""
검토 텍스트에서 다음 항목을 식별합니다.:\
- 리뷰어가 구매한 아이템
- 아이템을 만든 회사

리뷰는 세 개의 백틱으로 구분됩니다.\
"항목" 및 "브랜드"를 키로 사용하여 응답을 JSON 개체로 형식화합니다.
정보가 없으면 "unknown"을 값으로 사용합니다.
응답을 가능한 한 짧게 하십시오.
  
리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "항목": "램프",
    "브랜드": "루미나"
}


## Doing multiple tasks at once

In [9]:
prompt = f"""
검토 텍스트에서 다음 항목을 식별합니다.
- 감정(긍정적 또는 부정적)
- 리뷰어가 분노를 표현하고 있습니까? (참 또는 거짓)
- 리뷰어가 구매한 아이템
- 아이템을 만든 회사

리뷰는 세 개의 백틱으로 구분됩니다. \
"감정", "분노", "아이템" 및 "브랜드"를 키로 사용하여\
응답을 JSON 개체로 형식화합니다.
정보가 없으면 "unknown"을 값으로 사용합니다.
응답을 가능한 한 짧게 하십시오.
분노 값을 부울 형식으로 지정합니다.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "감정": "긍정적",
    "분노": false,
    "아이템": "램프",
    "브랜드": "루미나"
}


## Inferring topics

In [10]:
story = """
최근 정부가 실시한 설문조사에서 공공부문 직원들에게 자신이 근무하는 \
부서에 대한 만족도를 평가해 달라는 요청을 받았습니다.
그 결과 NASA가 가장 인기 있는 것으로 나타났다.
만족도 95%의 부서입니다.

NASA 직원인 John Smith는 이번 조사 결과에 대해 \
"NASA가 1위를 차지한 것이 놀랍지 않습니다.
놀라운 사람들과 놀라운 기회와 함께 일할 수 있는 훌륭한 장소입니다. 
이런 혁신적인 조직의 일원이 된 것을 자랑스럽게 생각합니다."

NASA 경영진도 결과를 환영했으며,톰 존슨(Tom Johnson) 이사는 \
“우리 직원들이 NASA에서의 업무에 만족하고 있다는 소식을 듣고 매우 기쁩니다.\
우리는 우리의 목표를 달성하기 위해 끊임없이 노력하는 \
재능 있고 헌신적인 팀을 보유하고 있으며, \
그들의 노력이 성과를 거두는 것을 보는 것은 환상적입니다."

또한 설문조사에 따르면 사회보장국은 직원의 45%만이 \
자신의 직업에 만족한다고 답해 만족도가 가장 낮은 것으로 나타났습니다. \
정부는 설문조사를 통해 직원들이 제기한 우려사항을 해결하고\
모든 부서에서 직무 만족도를 향상시키기 위해 노력할 것을 약속했습니다.
"""

## Infer 5 topics

In [11]:
prompt = f"""
세 개의 백틱으로 구분된 다음 텍스트에서 논의되고 있는 5개의 주제를 결정합니다.

각 항목을 한두 단어 길이로 만드세요.

응답을 쉼표로 구분된 항목 목록으로 형식화하십시오.

텍스트 샘플: '''{story}'''
"""
response = get_completion(prompt)
print(response)

- 공공부문 직원
- NASA
- John Smith
- Tom Johnson
- 사회보장국


In [12]:
response.split(sep=',')

['- 공공부문 직원\n- NASA\n- John Smith\n- Tom Johnson\n- 사회보장국']

In [13]:
prompt = f"""
세 개의 백틱으로 구분된 다음 텍스트에서 논의되고 있는 5개의 주제를 결정합니다.

각 항목을 한두 단어 길이로 만드세요.

응답을 쉼표로 구분된 list 로 형식화하세요.

텍스트 샘플: '''{story}'''
"""
response = get_completion(prompt)
print(response)

['설문조사', 'NASA', '만족도', '사회보장국', '직무 만족도']


## Make a news alert for certain topics

In [14]:
topic_list = [
"NASA", "지방 정부", "엔지니어링", "직원 만족도", "연방정부"
]
", ".join(topic_list)

'NASA, 지방 정부, 엔지니어링, 직원 만족도, 연방정부'

In [17]:
prompt = f"""
다음 주제 목록의 각 항목이 세 개의 백틱으로 구분된 \
아래 텍스트의 주제인지 확인합니다.

각 주제에 대해 0 또는 1의 목록으로 답변을 제공하십시오. \

다음과 같은 형태로:

항목:0 또는 1

주제 목록: {", ".join(topic_list)}

텍스트 샘플: '''{story}'''
"""
response = get_completion(prompt)
print(response)

NASA:1
지방 정부:0
엔지니어링:0
직원 만족도:1
연방정부:1


In [18]:
topic_dict = {i.split(':')[0]: int(i.split(':')[1]) for i in response.split(sep='\n')}
if topic_dict['NASA'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


## Try experimenting on your own!